In [14]:
import paramiko
# import numpy as np

# modify the outbound traffic scheduler / egress qdisc
# injecting the trasmitting latency for the linux kerner scheduler when engressing packages 

def apply_latency_to_node(node_ip, username, key_path, interface, latency, varied_delay):
    """Apply latency to a network interface on a node using SSH with a private key."""
    #- Delay of 300ms and random 50ms uniform variation with correlation value 25% (since network delays are not completely random)
    # ----before using 'change', be sure there's already a 'add' rule in each node. Eg, Add a 0ms delay rule first
    # sudo  tc qdisc add dev eth0 root netem delay 300ms 50ms 25%
    
    command_clear = f"sudo tc qdisc del dev {interface} root" # delete all existing rules first and then add, otherwise the add rules may fail.
    # as the test is running Melbourne Research Cloud (MRC) offers free on-demand computing resources to researchers at the University
    # the actual communication latency between each VM instance node is around 0.2 ~0.7 ms.
    # thus, we add a fixed latency between each node, with a variation (-/+ 2ms)
    command_add = f"sudo tc qdisc add dev {interface} root netem delay {0+latency}ms {0+varied_delay}ms 25%" 
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    client.load_system_host_keys()
    try:
        client.connect(node_ip, username=username, key_filename=key_path)
        stdin_clear, stdout_clear, stderr_clear = client.exec_command(command_clear)
        stdin_add, stdout_add, stderr_add = client.exec_command(command_add)

        print(stdout_add.read().decode())
    except Exception as e:
        print(f"Failed to apply latency to {node_ip}: {e}")
    finally:
        client.close()

def automate_latency_injection(node_details, latency = 0,  interface='eth0', varied_delay=0): # by default, set injected latency to 0ms
    """
    Apply latency to worker nodes using SSH keys for authentication.
    interface: Network interface to apply latency on.
    key_path: Path to the private SSH key.
    """

    # Build a scenario when each Node have  fixed slightly different delay to each other
    # in this scenario, "node chain(node pair)" delays are incrementaly different
    delta_dealy = 0 # 1 mill seconds
    node_count = 0
    
    for worker_name, worker_details in node_details.items():
        increce_dealy = delta_dealy*node_count # add delta incremental dealy, to create the scenario (0,2,4,6,...18)
        print(f"Applying {latency+increce_dealy}ms latency with {varied_delay}ms varied delay to {worker_name}")
        apply_latency_to_node(worker_details['ip'], worker_details['username'], worker_details['key_path'], interface, latency, varied_delay)
        node_count = node_count+1 # the nex node delay will add incrementaly

#node_details: Dictionary with node names as keys and details as values (IP, username, key path).
node_details = {
    'k8s-worker-2': {'ip': '172.26.132.91', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-3': {'ip': '172.26.133.31', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-6': {'ip': '172.26.133.55', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-7': {'ip': '172.26.130.22', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-8': {'ip': '172.26.130.82', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-9': {'ip': '172.26.133.118', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},

    
    'k8s-worker-1': {'ip': '172.26.128.30', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-3': {'ip': '172.26.133.31', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-4': {'ip': '172.26.132.241', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-5': {'ip': '172.26.132.142', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'}
    
    # Add more nodes as needed
}


'''
Steps of traffic control for the tc commands:
(1) 'delete' all existing rules:  sudo tc qdisc del dev eth0 root
(2) 'add' delay : sudo tc qdisc add dev eth0 root netem delay 20ms
(3) 'change' delay :  sudo tc qdisc change dev eth0 root netem delay 300ms 30ms 25%
(4) 'show' the existing rules: sudo tc qdisc show  dev eth0
'''




'''different latency injection scenarios, with different schemes'''
# (1) when test Istio service mesh overhead, Remove all injected delays
# automate_latency_injection(node_details, latency = -15, varied_delay=-10) # without any added extra dedaly

# (2) when test other scenarios, inject the fixed delays all the time.
automate_latency_injection(node_details, latency =0, varied_delay=-0)

# (3) when need to inject node coomunication anomolies, addjust "latency" and  "varied_delay".
# automate_latency_injection(node_details, latency =10, varied_delay=5)#



Applying 0ms latency with 0ms varied delay to k8s-worker-2

Applying 0ms latency with 0ms varied delay to k8s-worker-3

Applying 0ms latency with 0ms varied delay to k8s-worker-6

Applying 0ms latency with 0ms varied delay to k8s-worker-7

Applying 0ms latency with 0ms varied delay to k8s-worker-8

Applying 0ms latency with 0ms varied delay to k8s-worker-9

Applying 0ms latency with 0ms varied delay to k8s-worker-1

Applying 0ms latency with 0ms varied delay to k8s-worker-4

Applying 0ms latency with 0ms varied delay to k8s-worker-5



In [17]:
from matplotlib.cbook import print_cycles
import paramiko

# Global variable to keep track of the last used mark
last_mark = 0

def get_next_mark():
    """Generate the next unique mark."""
    global last_mark
    last_mark += 1
    return last_mark

def apply_latency_between_nodes(source_node_ip, destination_node_ip, username, key_path, interface, latency):
    """Apply latency between source and destination nodes using SSH with a private key."""
    mark_source = get_next_mark()
    mark_destination = get_next_mark()

    command_clear = f"sudo tc qdisc del dev {interface} root" # delete all existing rules first
    command_add = f"sudo tc qdisc add dev {interface} root handle 1: prio" # add root qdisc for prioritization
    command_mark_source = f"sudo iptables -t mangle -A OUTPUT -s {source_node_ip} -j MARK --set-mark {mark_source}"
    command_mark_destination = f"sudo iptables -t mangle -A OUTPUT -d {destination_node_ip} -j MARK --set-mark {mark_destination}"
    command_delay_source_to_dest = f"sudo tc qdisc add dev {interface} parent 1:{mark_source} handle {mark_source}: netem delay {latency}ms"
    command_delay_dest_to_source = f"sudo tc qdisc add dev {interface} parent 1:{mark_destination} handle {mark_destination}: netem delay {latency}ms"
    command_filter_source_to_dest = f"sudo tc filter add dev {interface} protocol ip parent 1:0 prio 1 handle {mark_source} fw flowid 1:{mark_source}"
    command_filter_dest_to_source = f"sudo tc filter add dev {interface} protocol ip parent 1:0 prio 1 handle {mark_destination} fw flowid 1:{mark_destination}"


    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    client.load_system_host_keys()
    try:
        client.connect(source_node_ip, username=username, key_filename=key_path)
        stdin_clear, stdout_clear, stderr_clear = client.exec_command(command_clear)
        stdin_add, stdout_add, stderr_add = client.exec_command(command_add)
        stdin_mark_source, stdout_mark_source, stderr_mark_source = client.exec_command(command_mark_source)
        stdin_mark_destination, stdout_mark_destination, stderr_mark_destination = client.exec_command(command_mark_destination)
        stdin_delay_source_to_dest, stdout_delay_source_to_dest, stderr_delay_source_to_dest = client.exec_command(command_delay_source_to_dest)
        stdin_delay_dest_to_source, stdout_delay_dest_to_source, stderr_delay_dest_to_source = client.exec_command(command_delay_dest_to_source)
        stdin_apply_filter_source_to_dest, stdout_apply_filter_source_to_dest, stderr_apply_filter_source_to_dest = client.exec_command(command_filter_source_to_dest)
        stdin_apply_filter_dest_to_source, stdout_apply_filter_dest_to_source, stderr_apply_filter_dest_to_source = client.exec_command(command_filter_dest_to_source)
        
        print(f"tested1, mark_source=:{mark_source}, mark_destination = {mark_destination}")

        print(stdout_delay_source_to_dest.read().decode())
        print(f"tested2, mark_source=:{mark_source}, mark_destination = {mark_destination}")
        print(stdout_delay_dest_to_source.read().decode())
        print(f"tested3, mark_source=:{mark_source}, mark_destination = {mark_destination}")
    except Exception as e:
        print(f"Failed to apply latency between {source_node_ip} and {destination_node_ip}: {e}")
    finally:
        client.close()

def automate_latency_injection(node_details, source_node_name1, source_node_name2, delay):
    """
    Apply latency between specified nodes using SSH keys for authentication.
    """
    source_ip1 = node_details[source_node_name1]['ip']
    source_ip2 = node_details[source_node_name2]['ip']
    username = node_details[source_node_name1]['username']  # Assuming username is the same for both nodes
    key_path = node_details[source_node_name1]['key_path']  # Assuming key_path is the same for both nodes
    interface = 'eth0'  # Assuming the interface name is eth0

    apply_latency_between_nodes(source_ip1, source_ip2, username, key_path, interface, delay)

# Execute the function with the node details dictionary
automate_latency_injection(node_details, 'k8s-worker-1', 'k8s-worker-2', 10)  # Example: Inject 10ms delay from node1 to node2
automate_latency_injection(node_details, 'k8s-worker-1', 'k8s-worker-3', 20)  # Example: Inject 20ms delay from node1 to node3
automate_latency_injection(node_details, 'k8s-worker-1', 'k8s-worker-4', 30)  # Example: Inject 30ms delay from node1 to node4


tested1, mark_source=:1, mark_destination = 2

tested2, mark_source=:1, mark_destination = 2

tested3, mark_source=:1, mark_destination = 2
tested1, mark_source=:3, mark_destination = 4

tested2, mark_source=:3, mark_destination = 4

tested3, mark_source=:3, mark_destination = 4
tested1, mark_source=:5, mark_destination = 6

tested2, mark_source=:5, mark_destination = 6

tested3, mark_source=:5, mark_destination = 6
